# 「Data Science」カーネルを起動

カーネル上ですべてのノートブックのインタラクションが実行されます。

# 画面右上の「No Kernel」をクリック
![](img/select_kernel.png)

# `Data Science` カーネルを選択
![](img/select_data_science_kernel.png)

# 画面右上からカーネルが起動したことを確認
![](img/confirm_kernel_started.png)

# 注意：　カーネルが起動するまで先に進まないでください
# \#\#\# カーネルが起動するまで待機してください！！！ \#\#\#

# セルをひとつひとつ実行するには `Shift+Enter` をタイプ

セルを選択して `Shift+Enter` をタイプすると出力が得られます。

# すべてのセルを実行するには、`Kernel` をクリック => `Restart Kernel and Run All Cells` を選択
![](img/restart-kernel-and-run-all-cells.png)

# ワークショップのイントロダクション

このワークショップはオライリージャパンから出版されている「実践 AWSデータサイエンス」に基づいています。

<!--[![Data Science on AWS](img/book_full_color_sm.png)](https://www.amazon.com/Data-Science-AWS-End-End/dp/1492079391/)-->
[![実践 AWSデータサイエンス](https://user-images.githubusercontent.com/6406410/136933305-65258bd6-cdb0-4148-a2a4-7b290e00800e.jpeg)](https://www.oreilly.co.jp/books/9784873119687/)

YouTubeビデオ、ミートアップ、本、コードはこちら:  **https://datascienceonaws.com**

# ワークショップについて

このハンズオンワークショップでは、Amazon SageMakerを使って自然言語処理のためのエンドツーエンドのAI/MLパイプラインを構築します。
言語表現の獲得には最先端の[BERT](https://arxiv.org/abs/1810.04805)モデルを使用して、テキストベースの商品レビューを分類するテキスト分類器をトレーニングおよびチューニングします。

BERTベースのNLPモデルを構築するために、1995年から2015年までの20年間にAmazon.comに寄せられた1億5千万件以上のカスタマーレビューを含む[Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)を使用します。
特に、`review_body`（自由形式のレビューテキスト）から `star_rating` （1が最低、5が最高の星評価）を予測する分類器をトレーニングします。

# 学習目標

このワークショップでは以下の内容について学びます。

* Amazon AthenaとParquetデータフォーマットを使用したS3へのデータの取り込み
* SageMakerノートブック上でのpandasやmatplotlibによるデータの可視化
* SageMaker Clarifyを使用したデータバイアス分析の実行
* Scikit-LearnおよびSageMaker Processing Jobを使用した生データセットに対する特徴量エンジニアリングの実行
* SageMaker Feature Storeを使用した特徴量の保存と共有
* TensorFlow、Keras、およびSageMaker Training Jobを使用したカスタムBERTモデルのトレーニングと評価
* SageMaker Processing Jobを使用したモデルの評価
* Amazon SageMaker ML Lineage Trackingを使用したモデルアーティファクトのトラッキング
* SageMaker Clarifyを使用したモデルのバイアスおよび説明可能性分析の実行
* SageMaker Model Registryを使用したモデルの登録とバージョン管理
* SageMaker Endpointを使用したREST推論エンドポイントへのモデルのデプロイ
* SageMaker Pipelinesを使用してエンドツーエンドのモデルパイプラインを構築し、MLワークフローのステップを自動化

# AmazonのAIおよび機械学習スタック
<img src="img/aws-ai-ml-stack.png" width="90%" align="left">

# Twitterで原著者をフォロー

In [ ]:
%%html

<a href="https://twitter.com/cfregly" class="twitter-follow-button" data-size="large" data-lang="en" data-show-count="false">Follow @cfregly</a>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

## _↑のボタンをクリック_

In [ ]:
%%html

<a href="https://twitter.com/anbarth" class="twitter-follow-button" data-size="large" data-lang="en" data-show-count="false">Follow @anbarth</a>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

## _↑のボタンをクリック_

# GitHubリポジトリにスターを押す

In [ ]:
%%html

<a class="github-button" href="https://github.com/oreilly-japan/data-science-on-aws-jp" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star oreilly-japan/data-science-on-aws-jp on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

## _↑のボタンをクリック_

# Data Science on AWSのウェブサイトにアクセス

In [ ]:
%%html

<iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# ユースケース: BERTモデルをファインチューニングし、テキスト分類器を作成

メインのデータセットとして [Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html) を選択しました。

- `marketplace`: 二文字の国コード（今回はすべて「US」）。
- `customer_id`: それぞれの書き手のレビュー集約に使われるランダムID。
- `review_id`: レビューのユニークID。
- `product_id`: Amazon標準識別番号（ASIN）。`http://www.amazon.com/dp/<ASIN>` が商品の詳細ページへのリンクとなる。
- `product_parent`: ASINの親。ひとつの親に対して複数のASINが存在しうる（同じ商品の色違いやフォーマット違いなど）。
- `product_title`: 商品のタイトル表記。
- `product_category`: レビューのグループ化に使う大まかな商品カテゴリー（このケースでは「デジタルビデオ」など）
- `star_rating`: レビューの星評価（1〜5）。
- `helpful_votes`: レビューへの「役に立った」投票の個数。
- `total_votes`: レビューへの全投票数。
- `vine`: レビューが[Vine](https://www.amazon.com/gp/vine/help)先取りプログラムの一環で書かれたか否か。
- `verified_purchase`: レビューが検証済みの購入に対するものか否か。
- `review_headline`: レビューそのもののタイトル。
- `review_body`: レビュー本体のテキスト。
- `review_date`: レビューが投稿された日付。

![BERT Training](img/bert_training.png)

BERTのアテンション機構はTransformerと呼ばれています。これは、偶然ではありませんが、[HuggingFace](https://github.com/huggingface/transformers)という会社によってメンテナンスされている、人気のBERT Pythonライブラリ「Transformer」の名前と同じです。今回は、より少ないメモリとコンピューティングパワーで利用できる、[DistilBert](https://arxiv.org/pdf/1910.01108.pdf)と呼ばれるBERTの変種を使用します。軽量ではありながら、今回扱うデータセットでも非常に優れた精度を維持します。

# BERTとSageMaker Pipeline

![](./img/bert_sagemaker_pipeline.png)

Processingステップでは、事前学習済みBERTモデルを使用して `review_body` テキストからBERT埋め込みを作成するための特徴量エンジニアリングを実行し、データセットをトレーニング、バリデーション、およびテストファイルに分割します。また、Tensorflowのトレーニングに最適化するために、ファイルをTFRecord形式で保存します。

Trainingステップでは、BERTモデルをカスタマーレビューデータセットに合わせてファインチューニングし、`review_body` に対する `star_rating` を予測するために新しい分類レイヤーを追加します。

Evaluationステップでは、トレーニングされたモデルとテストデータセットを入力とし、分類評価メトリクスを含むJSONファイルを生成します。

Conditionステップでは、Evaluationステップで測定されたモデルの精度がある閾値を超えた場合にこのモデルを登録する、という判断を行います。

# リソースの解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>